NumPy 提供了一个 nditer 迭代器对象，它可以配合 for 循环完成对数组元素的遍历。

In [1]:
import numpy as np

In [ ]:
# 使用 arange() 函数创建一个 3*4 数组，并使用 nditer 生成迭代器对象。

a = np.arange(0, 60, 5).reshape(3, 4)
# 使用nditer迭代器,并使用for进行遍历
for x in np.nditer(a):  # 迭代器
    print(x, end=" ")

0 5 10 15 20 25 30 35 40 45 50 55 

#### 遍历顺序
在内存中，Numpy 数组提供了两种存储数据的方式，分别是 C-order（行优先顺序）与 Fortrant-order（列优先顺序）。那么 nditer 迭代器又是如何处理具有特定存储顺序的数组呢？其实它选择了一种与数组内存布局一致的顺序，之所以这样做，是为了提升数据的访问效率。

在默认情况下，当我们遍历数组中元素的时候，不需要考虑数组的存储顺序，这一点可以通过遍历上述数组的转置数组来验证。

In [4]:
# a 的转置
b = a.T
b

array([[ 0, 20, 40],
       [ 5, 25, 45],
       [10, 30, 50],
       [15, 35, 55]])

In [ ]:
for x in np.nditer(b):  # 转置后的遍历
    print(x, end=", ")

0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 

从示例 1、2 的输出结果可以看出，a 和 a.T 的遍历顺序是一样的，也就是说，它们在内存中的存储顺序是一样的。

下面以 C 样式访问转置数组的副本。

In [6]:
for x in np.nditer(a.T.copy(order="C")):
    print(x, end=", ")

0, 20, 40, 5, 25, 45, 10, 30, 50, 15, 35, 55, 

通过示例 3 可知 a.T.copy(order = 'C') 的遍历结果与示例 1、2 的数组遍历结果不一样。究其原因，就是因为它们在内存中的存储方式不一样。

order 是 numpy.ndarray.copy() 方法的一个参数。order 参数指定了多维数组的存储顺序。它有两个可选的值：

- 'C' (C-style row-major order)：
    - 数据按行的顺序存储，即第一维的元素存储在内存中相邻的位置，第二维的元素按行访问。这是默认的存储顺序。
- 'F' (Fortran-style column-major order)：
    - 数据按列的顺序存储，即第一维的元素按列访问。这种存储顺序通常用于 Fortran 数学库。

#### 指定遍历顺序
通过 nditer 对象的order参数来指定数组的遍历的顺序

In [7]:
a

array([[ 0,  5, 10, 15],
       [20, 25, 30, 35],
       [40, 45, 50, 55]])

In [ ]:
for x in np.nditer(a, order="C"):
    print(x, end=", ")

0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 

In [ ]:
for x in np.nditer(a, order="F"):
    print(x, end=", ")

0, 20, 40, 5, 25, 45, 10, 30, 50, 15, 35, 55, 

#### 修改数组元素值
nditer 对象提供了一个可选参数op_flags，它表示能否在遍历数组时对元素进行修改。它提供了三种模式，如下所示：
1) read-only
只读模式，在这种模式下，遍历时不能修改数组中的元素。
2) read-write
读写模式，遍历时可以修改元素值。
3) write-only
只写模式，在遍历时可以修改元素值。

In [10]:
a

array([[ 0,  5, 10, 15],
       [20, 25, 30, 35],
       [40, 45, 50, 55]])

In [12]:
for x in np.nditer(a, op_flags=["readwrite"]):
    x[...] = 2 * x

a

array([[  0,  10,  20,  30],
       [ 40,  50,  60,  70],
       [ 80,  90, 100, 110]])

#### 外部循环使用
nditer 对象的构造函数有一个“flags”参数，它可以接受以下参数值（了解即可）：

flags参数说明
|参数值	            |描述说明|
|:--:|:--:|
|c_index	        |可以跟踪 C 顺序的索引。|
|f_index	        |可以跟踪 Fortran 顺序的索引。|
|multi_index	    |每次迭代都会跟踪一种索引类型。|
|external_loop	    |返回的遍历结果是具有多个值的一维数组。|


In [13]:
a

array([[  0,  10,  20,  30],
       [ 40,  50,  60,  70],
       [ 80,  90, 100, 110]])

In [15]:
for x in np.nditer(a, flags=["external_loop"], order="F"):
    print(x)

[ 0 40 80]
[10 50 90]
[ 20  60 100]
[ 30  70 110]


#### 迭代多个数组
如果两个数组都能够被广播，那么 nditer 对象就可以同时对它们迭代。

假设数组 a 的维度是 3*4，另一个数组 b 的维度是 1*4 （即维度较小的数组 b 可以被广播到数组 a 中）

In [16]:
a

array([[  0,  10,  20,  30],
       [ 40,  50,  60,  70],
       [ 80,  90, 100, 110]])

In [ ]:
b = np.array([1, 2, 3, 4], dtype=int)
b

array([1, 2, 3, 4])

In [ ]:
# 广播迭代
for x, y in np.nditer([a, b]):
    print("%d:%d" % (x, y), end=", ")

0:1, 10:2, 20:3, 30:4, 40:1, 50:2, 60:3, 70:4, 80:1, 90:2, 100:3, 110:4, 